In [1]:
## Import packages
import os
import csv
import re
import random
import math
import wave
import numpy as np
import pandas as pd
from scipy.io import wavfile
from collections import Counter

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Parameters voor het maken van tf-records
audio_embedding_feature_name = 'audio_embedding'
pca_params = 'models/vggish_pca_params.npz'
checkpoint = 'models/vggish_model.ckpt'
yt_checkpoint = 'models/youtube_model.ckpt'

In [3]:
## Lees alle csv-files in
bal_labels = pd.read_csv('csv_files/balanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

unbal_labels = pd.read_csv('csv_files/unbalanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

eval_labels = pd.read_csv('csv_files/eval_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

In [4]:
## Map waarin yt-wavjes staan
wavfile_path = "wav_files"

## Geef aan of de tf-records voor jungle of urban moeten worden gedownload (ik neem aan dat we alleen jungle doen)
target = "jungle" #"urban"
mid_to_label = pd.read_csv("csv_files/class_labels_indices_" + target + "_prop.csv", sep=";")

## embeddings komen nu in 1 groot tf-record te staan
## We kunnen ook een counter maken: na elke 1000, maak nieuwe tf-record
tfrecord_path = "tfrecords/added_data/"

In [5]:
bal_labels.head

<bound method NDFrame.head of               YTID  start_seconds  end_seconds  \
0      --PJHxphWEs           30.0         40.0   
1      --ZhevVpy1s           50.0         60.0   
2      --aE2O5G5WE            0.0         10.0   
3      --aO5cdqSAg           30.0         40.0   
4      --aaILOrkII          200.0        210.0   
5      --cB2ZVjpnA           30.0         40.0   
6      --ekDLDTUXA           30.0         40.0   
7      -0DLPzsiXXE           30.0         40.0   
8      -0DdlOuIFUI           50.0         60.0   
9      -0FHUc78Gqo           30.0         40.0   
10     -0O3e95y4gE          100.0        110.0   
11     -0SdAVK79lg           30.0         40.0   
12     -0Vl4HyWRk8          410.0        420.0   
13     -0mG4W5Hlq8          270.0        280.0   
14     -0mjrMposBM           80.0         90.0   
15     -11LhdJgBb8           30.0         40.0   
16     -1LrH01Ei1w           30.0         40.0   
17     -1TLtjPtnms           10.0         20.0   
18     -1iKLvsRBbE  

In [6]:
mid_to_label

,index,mid,display_name,total,target_ind
0,0,/t/dd00134,"Car, boat, motorcyle, truck",30189,0.083
1,1,/m/01j4z9,Chainsaw,1787,1.400
2,2,/m/07pczhz,Chop,434,3.000
3,3,/m/032s66,"Gunshot, gunfire",4221,0.590
4,4,/m/09ct_,Helicopter,3698,0.680
5,5,/t/dd00003,Male singing,7224,0.350
6,6,/m/07rpkh9,Cattle,3347,0.750
7,0,/m/02rlv9,"Car, boat, motorcyle, truck",30189,0.083
8,0,/m/04_sv,"Car, boat, motorcyle, truck",30189,0.083
9,7,/m/06bz3,Radio,4214,0.590


In [7]:
## Lijst van alle labels (nieuwe) en alle combi's
all_labels = []

## Proportie train/test
pct_train = 0.6

In [8]:
## Voor alle wav-files in een map, gebeurt het volgende:

## Eerst wordt het wav-file ingelezen en de bijbehorende label(s) opgezocht in een csv-bestand
## Als video-id wordt een willekeurig id'tje gegeven (in dit geval allemaal dezelfde)

## Dan parsen we wav-file naar embeddings: 
# (dit gebeurt door vggish_input.wavfile_to_example aan te roepen)
# Stap 1a: lezen van wav-file, input is array met samples die db aanduiden. Ook sample rate (per sec) wordt gelezen
# Stap 1b: Bij 2d array (stereo, ipv mono) bereken gemiddelde, daarna normaliseren (delen door 32.768)
# Stap 2: Bepaal examples in vorm [batch size, num frames, num bands].
    # Hierbij worden voor verschillende batches (omdat alles tegelijk niet in 1x in NN kan),
    # een log mel spectrogram gemaakt (in vorm [num_frames, num_bands])
# Stap 3: Bepaal features: nu wordt de embedding laag gemaakt (PCA-components, discreet maken etc)
    # Hiervoor worden model-parameters opgehaald die eerder zijn opgeslagen
    
## Daarna wordt een sequence example gemaakt (in getSequenceExample) en het als tf-records weggeschreven

In [9]:
## Function that takes examples from wav-file as input and returns a sequence example

def getSequenceExample(examples_batch, labels, video_id=[b'-1LrH01Ei1w']):
    with tf.Graph().as_default(), tf.Session() as sess:

        # Prepare a postprocessor to munge the model embeddings.
        pproc = vggish_postprocess.Postprocessor(pca_params)
    
        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)
        
        # Run inference and postprocessing.
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})
        print(embedding_batch.shape)
        
        postprocessed_batch = pproc.postprocess(embedding_batch)
        #print(postprocessed_batch)


        ## Maak labels en video-id voor in de example
        label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
        videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))
        
        ## Maak sequence example
        seq_example = tf.train.SequenceExample(
            context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
        
    return(seq_example)

In [10]:
## Deze functie checkt of een example wordt gemaakt (downsamplen)
def checkIfNewExample(labels):
    
    ## Alleen bij 1 label worden er examples overgeslagen (of bij write_all_examples)
    if(len(labels)>1):
        return True
    else:
        label = list(labels)[0]
        prop = mid_to_label.loc[mid_to_label['index']==np.int(label), "target_ind"].values[0]
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:
            return True
        else:
            return False
    return False

In [11]:
## Deze functie checkt of er nog een example moet worden gemaakt (upsamplen)
## Momenteel kan het aantal enkel verdubbeld worden

def checkIfExtraExample(labels):
    
    ## Alleen bij 1 label worden er examples gekopieerd (of als er niet (up)gesampled wordt)
    if(len(labels)>1):
        return False
    else:
        label = list(labels)[0]
        prop = mid_to_label.loc[mid_to_label['index']==np.int(label), "target_ind"].values[0]-1
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:    
            return True
        else:
            return False
    return False

In [12]:
def getLabels(mid_str):
    ## Maak lijst van m-id's
    mid_list = mid_str.split(',')
    labels = []
    
    ## Voor elk m-id, vind labels, (if any labels: add to label list)
    for mid in mid_list:
        if (mid_to_label.loc[mid_to_label["mid"] == mid, "mid"].any()):
            labels.append(mid_to_label.loc[mid_to_label["mid"] == mid, "index"].values[0])
    
    ## Return unique set of labels
    return set(labels)

In [13]:
def getStarttime(vid_to_mid, ytid):
    start_time = vid_to_mid.loc[vid_to_mid["YTID"] == ytid, "start_seconds"].values[0]
    return start_time

In [14]:
def readWav(vid_to_mid, folder):
    
    ## Maak tf-records aan
    train_tfrecord = str(tfrecord_path + folder + '_train.tfrecord')
    test_tfrecord = str(tfrecord_path + folder + '_test.tfrecord')
    
    train_writer = tf.python_io.TFRecordWriter(train_tfrecord)
    test_writer = tf.python_io.TFRecordWriter(test_tfrecord)
    
    ## Read and rewrite all test files
    files = gfile.Glob(str(wavfile_path + "/" + folder + "/*.wav"))
  
    ## Count om bij te houden hoe ver hij is
    count=0
    for file in files:
    
        count+=1
        print("\nfile nr: " + str(count))
        filename = file.split("\\")[-1]
        if(re.match("wav_files",filename)):
            filename = file.split("/")[-1]
        filename = filename[0:-4]
        ytid = filename.split("_^_")[0].replace("vid", "")
        print("ytid: " + str(ytid))
        #start = getStarttime(vid_to_mid, ytid)
        
        ## Vind bijbehorende m-ids
        mids = vid_to_mid.loc[vid_to_mid["YTID"] == ytid, "positive_labels"].values[0]
        new_labels = getLabels(mids)
        
     
        ## This function reads the wav file and converts the samples into np arrays of [batch size, num frames, num bands]
        examples_batch = vggish_input.wavfile_to_examples(file)

        ## Bij meer dan 10 sec: neem middelste 10 sec
        if(examples_batch.shape[0]>10):
            nr_sec = examples_batch.shape[0]
            start = int(math.floor((nr_sec-10)/2))
            end = int(nr_sec-math.ceil((nr_sec-10)/2))
            examples_batch = examples_batch[start:end, :, :]

        ## Schrijf alleen bij meer dan 5 sec
        ## Check if new example: voor downsamplen
        #if(examples_batch.shape[0]>5 and checkIfNewExample(new_labels)):
        if(examples_batch.shape[0]>5):
            all_labels.extend(new_labels)

            seq_example = getSequenceExample(examples_batch, new_labels)
            rand = random.random()
            if rand <= pct_train:
                train_writer.write(seq_example.SerializeToString())
            else:
                test_writer.write(seq_example.SerializeToString())

            ## Voor upsamplen
            if(checkIfExtraExample(new_labels)):
                rand = random.random()
                if rand <= pct_train:
                    train_writer.write(seq_example.SerializeToString())
                else:
                    test_writer.write(seq_example.SerializeToString())

                all_labels.extend(new_labels)

    train_writer.close()
    test_writer.close()

In [17]:
#readWav(bal_labels, "bal")
#readWav(unbal_labels, "unbal")
readWav(eval_labels, "eval")


file nr: 1
ytid: -1II0Di9Hkc
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 2
ytid: -1OlgJWehn8
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 3
ytid: -23CeprtibU
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 4
ytid: -2EKWgTNEYU
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 5
ytid: -2sE5CH8Wb8
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 6
ytid: -3rHVsIj1M8
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 7
ytid: -8C-gydUbR8
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 8
ytid: -BY64_p-vtM
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 9
ytid: -ct58kOlLfs
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 10
ytid: -CZ1LIc8aos
INFO:t

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(9, 128)

file nr: 78
ytid: 1xhSBniwGbM
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 79
ytid: 1XO0SqsZhHU
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 80
ytid: 1YFa9CpH7Kc
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 81
ytid: 1ZEzs9zmZaU
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 82
ytid: 1ZHnWpLQtyo
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 83
ytid: 2-Yz378XB2Q
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 84
ytid: 209LviaOJL0
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 85
ytid: 20Nlj7Cz4E0
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 86
ytid: 26jTWRMRoxY
INFO:tensorflow:Restoring par

(10, 128)

file nr: 153
ytid: 4Tpy1lsfcSM
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 154
ytid: 4W8IZxn3nEI
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 155
ytid: 4yPdhTHZqLg
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 156
ytid: 4ZSad2HmTJE
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 157
ytid: 53KoRxxr1jc
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 158
ytid: 566i3rpolg8
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 159
ytid: 56HqpC_KJhE
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 160
ytid: 5DjZHCumLfs
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(9, 128)

file nr: 161
ytid: 5dLgZwcWe8Q
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 

(10, 128)

file nr: 230
ytid: 8OKAwDv20R8
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(9, 128)

file nr: 231
ytid: 8RjJvfW0Q3M
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 232
ytid: 8rmoYAfr0t8
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 233
ytid: 8sd513xQzV4
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 234
ytid: 8UNCaKU0O4A
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 235
ytid: 9-8P2AsfmzU
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 236
ytid: 9-tUObAs28M
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 237
ytid: 92WhvpMzkGY
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 238
ytid: 9EsgN-WS2qY
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 306
ytid: B8_Y9qmHFWQ
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 307
ytid: B98yOZSj2GE
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(9, 128)

file nr: 308
ytid: ba43-gH05H4
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 309
ytid: BBukw6JpCeg
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 310
ytid: bciLCctZfjE
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 311
ytid: BCzUvCRiBZs
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 312
ytid: Bdz-NtAD2SU
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 313
ytid: Bf3aAtNK8Sk
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 314
ytid: bHYb3-Qpmcc
INFO:tensorflow:Rest

...

(10, 128)

file nr: 683
ytid: MWTTe0M9vi4
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 684
ytid: mwZ__VNBuJU
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 685
ytid: MY0PsDE3xHs
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 686
ytid: myy-OtJbCfQ
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 687
ytid: mZIBXZlICgw
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 688
ytid: n-A3BJEpmz8
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 689
ytid: N-_m-dYt6NA
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 690
ytid: N0d1cK2k1Eg
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 691
ytid: N9C2AYVnas4
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr:

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 759
ytid: PpK09wcJWVE
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 760
ytid: PrWy40DYZQU
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 761
ytid: ps8ulH55E5Q
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 762
ytid: PSt0xAYgf4g
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 763
ytid: pt-J_L-OFI8
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 764
ytid: pTRiK6Cc9fo
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 765
ytid: pumgHCBx9iY
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 766
ytid: Q1CMSV81_ws
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 767
ytid: q1gQRZ3Hi1I
INFO:tensorflow:Res

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 835
ytid: sbpW3Z87Nbc
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 836
ytid: sCXkuo5_s4M
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 837
ytid: sfdLrmxahRs
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 838
ytid: SGFYFPs3Fic
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 839
ytid: SiBIYAiIajM
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 840
ytid: SJN5vcGGF4I
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 841
ytid: SkbFczIabRY
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 842
ytid: skxOyyYReQk
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 843
ytid: sKZFrqpLb18
INFO:tensorflow:Res

(10, 128)

file nr: 910
ytid: V2OjPsg7ydY
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 911
ytid: v5fePy2J1XU
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 912
ytid: v94qdpJqK9w
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 913
ytid: V9txnWLjP9k
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 914
ytid: vauFW9Nys8Q
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 915
ytid: vb1SalyslCs
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 916
ytid: Vbc-veezx0w
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 917
ytid: Vdh_JIJCLxA
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr: 918
ytid: vdJrCVU8gHE
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
(10, 128)

file nr:

Error: unknown format: 65534

In [16]:
## Print voor alle labels het aantal voorkomens in de nieuwe tf-records

sum_occur = 0
cnt_labels = Counter(all_labels)

for i in cnt_labels.most_common(100):
    sum_occur = sum_occur + i[1]
    print(mid_to_label.loc[mid_to_label['index']==i[0]]['display_name'].values[0], ' : ', str(i[1]))

Car, boat, motorcyle, truck  :  248
Gunshot, gunfire  :  166
Wind noise (microphone)  :  140
Chop  :  116
Cattle  :  112
Shout  :  104
Chainsaw  :  83
Male singing  :  58
Radio  :  57
Silence  :  57
Helicopter  :  56
